In [1]:
from keras.layers import Input, Lambda, Dense, Flatten, BatchNormalization,Activation
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE=[224,224]

In [3]:
train_path=r"C:\Users\Nitro\Desktop\MajorProject\Soybean"
valid_path=r"C:\Users\Nitro\Desktop\MajorProject\testdata"

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
for layer in vgg.layers:
  layer.trainable = False

In [6]:
folders = glob(r"C:\Users\Nitro\Desktop\MajorProject\Soybean\*")

In [7]:
x = Flatten()(vgg.output)
x = Dense(516, activation='relu')(x)
x= BatchNormalization()(x)
x= Dense(256)(x)
x = BatchNormalization()(x)  # Batch Normalization
x = Activation('relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)



In [8]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
# tell the model what cost and optimization method to use
import tensorflow as tf
adadelta_optimizer = tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)
model.compile(
 loss='categorical_crossentropy',
 optimizer='adam',
 metrics=['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1/255,validation_split=0.2,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True)



training_set = datagen.flow_from_directory(r"C:\Users\Nitro\Desktop\MajorProject\Soybean",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')

test_set =datagen.flow_from_directory(r"C:\Users\Nitro\Desktop\MajorProject\testdata",
                                        target_size = (224, 224),
                                        batch_size = 32,
                                        class_mode = 'categorical',subset='validation')

Found 4416 images belonging to 6 classes.
Found 1069 images belonging to 6 classes.


In [11]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\Nitro\AppData\Local\Temp\ipykernel_17196\2033252562.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
138/138 [==============================] - 553s 4s/step - loss: 0.8929 - accuracy: 0.6544 - val_loss: 0.9139 - val_accuracy: 0.6717
Epoch 2/10
138/138 [==============================] - 552s 4s/step - loss: 0.6196 - accuracy: 0.7654 - val_loss: 1.7821 - val_accuracy: 0.5201
Epoch 3/10
138/138 [==============================] - 542s 4s/step - loss: 0.5108 - accuracy: 0.8111 - val_loss: 1.3755 - val_accuracy: 0.6043
Epoch 4/10
138/138 [==============================] - 539s 4s/step - loss: 0.4417 - accuracy: 0.8361 - val_loss: 0.7364 - val_accuracy: 0.7371
Epoch 5/10
138/138 [==============================] - 538s 4s/step - loss: 0.4131 - accuracy: 0.8424 - val_loss: 0.9021 - val_accuracy: 0.6866
Epoch 6/10
138/138 [==============================] - 534s 4s/step - loss: 0.3527 - accuracy: 0.8693 - val_loss: 0.6508 - val_accuracy: 0.7774
Epoch 7/10
138/138 [==============================] - 540s 4s/step - loss: 0.3466 - accuracy: 0.8664 - val_loss: 0.6694 - val_accuracy: 0.7652

In [12]:
# model.save('vgg2clf.h5')
# from keras.models import load_model
# saved_model=load_model("vgg2clf.h5")
# from tensorflow.keras.models import load_model import numpy as np

# from sklearn.metrics import classification_report, confusion_matrix
# #Make predictions on the validation data

# validation_predictions= saved_model.predict(test_set)

# # Convert one-hot encoded Labels to class indices

# y_true = test_set.classes

# y_pred = np.argmax(validation_predictions, axis=1)


# # Classification Report

# class_report =classification_report(y_true, y_pred, target_names= training_set.class_indices.keys())

# print("Classification Report:\n", class_report)

# #Confusion Matrix confusion_matrix(y_true, y_pred) conf_matrix confusion_mat
# conf_matrix = confusion_matrix(y_true, y_pred)

# print("Confusion Matrix:\n", conf_matrix)
from tensorflow.keras.models import load_model
import numpy as np

model.save('vgg2clf.h5')
saved_model = load_model("vgg2clf.h5")

from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the validation data
validation_predictions = saved_model.predict(test_set)

# Convert one-hot encoded Labels to class indices
y_true = test_set.classes
y_pred = np.argmax(validation_predictions, axis=1)

# Classification Report
class_report = classification_report(y_true, y_pred, target_names=training_set.class_indices.keys())
print("Classification Report:\n", class_report)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)


34/34 [==============================] - 274s 8s/step
Classification Report:
                                      precision    recall  f1-score   support

                    Broken soybeans       0.17      0.15      0.16       198
                  Immature soybeans       0.20      0.20      0.20       223
                    Intact soybeans       0.19      0.22      0.20       223
Part of the original soybean images       0.00      0.00      0.00         1
              Skin-damaged soybeans       0.20      0.21      0.20       223
                   Spotted soybeans       0.18      0.16      0.17       201

                           accuracy                           0.19      1069
                          macro avg       0.16      0.16      0.16      1069
                       weighted avg       0.19      0.19      0.19      1069

Confusion Matrix:
 [[29 41 46  0 53 29]
 [40 45 53  0 43 42]
 [38 44 49  1 49 42]
 [ 0  0  1  0  0  0]
 [34 53 56  0 46 34]
 [32 40 54  0 42 33]]


In [13]:
#prediction
import keras.utils as image
img=(r"C:\Users\Nitro\Downloads\Photo.jpg")
img1=image.load_img(img,target_size=(224,224))
img1_array=image.img_to_array(img1)
img1_array=np.expand_dims(img1_array,axis=0)
img1_array/=255.0



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Nitro\\Downloads\\Photo.jpg'

In [ ]:
output=saved_model.predict(img1_array)
class_indices=training_set.class_indices
predicted_class=np.argmax(output,axis=1)
predicted_label=[k for k,v in class_indices.items() if v==predicted_class][0]
print("predicted class",predicted_label)
print("predicted class",output)


In [ ]:
max_idx=np.argmax(output)

max_idx
